<a href="https://colab.research.google.com/github/dellaert/robotics2/blob/main/S16_sorter_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [20]:
# %pip install - q - U gtbook


In [21]:
import numpy as np
import pandas as pd
import gtsam

import plotly.express as px
import plotly.graph_objs as go

import plotly.io as pio
pio.renderers.default = "notebook"

from gtbook.discrete import Variables
from gtbook.display import pretty, show

# recap from S11:
variables = Variables()
categories = ["cardboard", "paper", "can", "scrap metal", "bottle"]
Category = variables.discrete("Category", categories)


# Learning


## Overview

* S11: estimate prior from counts, add "bogus counts" to "smooth"
* S13, discrete: estimate sensor mode from counts, possibly smooth
* S13, continuous: fit mean and standard deviation (/N-1)


## Estimating a Discrete PMF

> : Count the observations.


The normalized histogram is a good approximation of the PMF. The more samples that go in, the better the approximation. For example, let us assume that, at a different cell, we observe the cell for a while and note what category each piece of trash is in. We might see something like:


In [22]:
data = [1, 1, 1, 2, 1, 1, 1, 3, 0, 0, 0, 1,
        2, 2, 2, 2, 4, 4, 4, 1, 1, 2, 1, 2, 1]


Using plotly express, we can easily plot a histogram


In [23]:
px.histogram(x=data)


We can get the counts from numpy as well, and then estimate the probability of each category simply by divinding by the number of data points:


In [24]:
counts, _ = np.histogram(data, bins=5)
estimated_pmf = counts/sum(counts)
print(f"Counts: {counts}\nEstimated PMF: {estimated_pmf}")


Counts: [ 3 11  7  1  3]
Estimated PMF: [0.12 0.44 0.28 0.04 0.12]


We can now easily turn it into a GTSAM discrete prior instance:


In [25]:
# catgeory_prior = gtsam.DiscretePrior(Category, estimated_pmf) # bug in 4.2a0 !
prior = gtsam.DiscretePrior(Category, "0.12/0.44/0.28/0.04/0.12")
pretty(prior, variables)


 *P(Category)*:

|Category|value|
|:-:|:-:|
|cardboard|0.12|
|paper|0.44|
|can|0.28|
|scrap metal|0.04|
|bottle|0.12|


Note that the counts are the only quantitities one needs to estimate a PMF: a statistication would say that the counts are a **sufficient statistic** for the purpose of estimating the probability distribution. In fact, GTSAM can just take the counts themselves, as it normalizes internally:


In [26]:
prior = gtsam.DiscretePrior(Category, "3/11/7/1/3")
pretty(prior, variables)


 *P(Category)*:

|Category|value|
|:-:|:-:|
|cardboard|0.12|
|paper|0.44|
|can|0.28|
|scrap metal|0.04|
|bottle|0.12|


### Smoothing

> : We make up fake data to deal with sparse data.


Another trick that statisticians and machine learning practicioners employ is "smoothing", which is especially important when you have very little data. Indeed, sometimes a category value will get zero counts, even though you *know* that occasionally we wil get that category. **Smoothing* is the process of letting the estimator know this by adding "pseudo-counts". For example, a very common approach is to simply add 1 to every count:


In [27]:
counts, _ = np.histogram(data, bins=5)
smoothed_counts = counts + 1
smoothed_pmf = smoothed_counts/sum(smoothed_counts)


Comparing the two, we see that the smoothed PMF is more uniform, and accords more probability "undercounted" categories than the raw counts:


In [28]:
# Some pandas magic to display a nice side-by-side table:
df = pd.DataFrame(
    {"raw": estimated_pmf, "smoothed": smoothed_pmf}, index=categories)
df


,raw,smoothed
cardboard,0.12,0.133333
paper,0.44,0.400000
can,0.28,0.266667
scrap metal,0.04,0.066667
bottle,0.12,0.133333


In [29]:
# and some plotly magic to plot them side by side:
fig = px.bar(df, x=categories, y="raw", barmode="group", title="Smoothed PMF")
fig.add_bar(x=categories, y=smoothed_pmf, name="smoothed")


## Modeling a Sensor from Data

> : When learning a conditional distribution, we need to separate out the counts based on the conditioning variable.


A `gtsam.DiscreteConditional` takes the counts, grouped by the conditioning variable. In our case, `Category` can take on 5 separate values, and hence we have five groups. For example a binary sensor:


In [30]:
Conductivity = variables.binary("Conductivity")
P_Conductivty_Category = gtsam.DiscreteConditional(
    Conductivity, [Category], "80/20 40/60 12/4 100/150 10/10")
pretty(P_Conductivty_Category, variables)


 *P(Conductivity|Category)*:

|*Category*|false|true|
|:-:|:-:|:-:|
|cardboard|0.8|0.2|
|paper|0.4|0.6|
|can|0.75|0.25|
|scrap metal|0.4|0.6|
|bottle|0.5|0.5|


And for a three-valued sensor:


In [31]:
ThreeValued = variables.discrete("ThreeValued", ["Value1", "Value2", "Value3"])
P_ThreeValued_Category = gtsam.DiscreteConditional(ThreeValued, [Category],
                                                   "10/70/20 20/20/60 2/10/4 100/100/50 5/5/10")
pretty(P_ThreeValued_Category, variables)


 *P(ThreeValued|Category)*:

|*Category*|Value1|Value2|Value3|
|:-:|:-:|:-:|:-:|
|cardboard|0.1|0.7|0.2|
|paper|0.2|0.2|0.6|
|can|0.125|0.625|0.25|
|scrap metal|0.4|0.4|0.2|
|bottle|0.25|0.25|0.5|


Once again, note that the *rows* are normalized to be proper PMFs, given the conditioning variable. The columns are not, and instead form likelihoods over the category, when a particular value is observed for `ThreeValued` .

Again, we can add pseudo-counts to all counts if we have very little data, or to specific groups if you only have prior knowledge about a particular setting. If you really know nothing about the behavior of a sensor, adding a pseudo-count of 1 is a good thing to do in general: it prevents according zero probability to a rare event. The downside is that you will have a biased view of the CPT, but this disadvantage quickly goes away as you add more and more data.


## Fitting a Gaussian

If we observe *continuous* data that we suspect is generated from a Gaussian density, then we can easily fit the estimated mean $\hat{\mu}$ by $$\hat{\mu} = \frac{1}{N} \sum_i x_i.$$ 

The other statistic we need is the variance $\sigma^2$ defined as the expectation of the squared error:$$E[(x-\mu)^2].$$

Fitting the variance can be done after we obtained the empirical mean $\hat{\mu}$, by
$$\widehat{\sigma^2} = \frac{1}{N-1} \sum_i (x_i-\hat{\mu})^2.$$ 

The standard deviation is defined as the square root of the variance, and hence: $$\widehat{\sigma} = \sqrt{\widehat{\sigma^2}}.$$


**Note**: above we divide by $N-1$ and not by $N$: informally, the reason is that we already "used up" one data point by estimating the mean from our samples, and we correct for that to get an "unbiased" estimate for the variance.


Below is some python code to do just that, using the `numpy` library. Let us first generate some "data" from a Gaussian with known mean and standard deviation:


In [32]:
mean = 200  # grams, say...
stddev = 50  # also in grams
N = 200  # number of samples
data = np.random.normal(mean, stddev, N)


When we plot a histogram, we can see the typical "bell curve" shape emerge (try increasing N!):


In [33]:
nbins = N//10
px.histogram(x=data, nbins=nbins)


The sample mean is easy enough with `np.mean` :


In [34]:
mu_hat = np.mean(data)
mu_hat


196.70751384542132

In [35]:
np.var(data, ddof=1)


2196.004917286482

Of course we can do this by ourselves as well:


Note that though with 200 samples the histogram is quite "messy", the sample mean $\hat{\mu}$ is close to the true mean $\mu=200$.

There is also a function `np.var` that calculates the sample variance, but we need to take care to provide the `ddof=1` argument to get the unbiased estimate (do `help(np.var)` to find out more).


Here is the code for variance:


In [36]:
var_hat = np.sum(np.square(data-mu_hat))/(N-1)
var_hat


2196.004917286482

By taking the square root, we see that it matches our ground truth standard deviation $\sigma$ quite well:


In [37]:
sigma_hat = np.sqrt(var_hat)
print(sigma_hat)


46.8615505215788


### Comparison

We can now plot our estimated Gaussian together with the data:


In [38]:
fig = px.histogram(x=data, nbins=nbins)
X = np.arange(0, 350)
K = N*(400/nbins)/np.sqrt(2*np.pi*var_hat)  # expected height of histogram...
fig.add_trace(go.Scatter(x=X, y=K * np.exp(-0.5*np.square(X-mu_hat)/var_hat)))
fig
